In [ ]:
import os
import numpy as np
import csv
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.filters import sobel
from skimage.feature import hog, local_binary_pattern
from skimage.measure import moments_hu
from skimage import img_as_ubyte
from tqdm import tqdm
import tifffile as tiff  # Importar tifffile para leer imágenes TIFF

# Directorios de las imágenes y las máscaras
directorio_satelital = "roads/sat"
directorio_gt = "roads/gt"

# Emparejar las imágenes con sus máscaras (suponiendo que tienen el mismo nombre)
pares_imagenes_mascaras = list(zip(sorted(os.listdir(directorio_satelital)), sorted(os.listdir(directorio_gt))))

# Por ejemplo, si las imágenes están en .tiff, ajusta la extensión
pares_imagenes_mascaras = [
    (img, img.replace('.tiff', '.tif'))  # Suponiendo que las máscaras tengan la extensión .tif
    for img in sorted(os.listdir(directorio_satelital))
]
print(f"Se encontraron {len(pares_imagenes_mascaras)} pares de imágenes y máscaras.")

# Nombre del archivo CSV
csv_file = "caracteristicas_imagenes.csv"

# Abrir el archivo CSV en modo escritura
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Escribir los encabezados del CSV
    writer.writerow(['sobel', 'hog', 'lbp', 'hu_moments_1', 'hu_moments_2', 'hu_moments_3', 'hu_moments_4', 'hu_moments_5', 'hu_moments_6', 'hu_moments_7', 'etiqueta', 'imagen_idx'])
    
    # Cargar las imágenes y máscaras
    for imagen_idx, (imagen_nombre, mascara_nombre) in tqdm(enumerate(pares_imagenes_mascaras), desc="Cargando imágenes y máscaras"):
        # Leer la imagen satelital
        ruta_imagen = os.path.join(directorio_satelital, imagen_nombre)
        imagen = imread(ruta_imagen)
        
        # Leer la máscara de ground truth (con tifffile para imágenes TIFF)
        ruta_mascara = os.path.join(directorio_gt, mascara_nombre)
        mascara = tiff.imread(ruta_mascara)
        
        # Asegurarse de que la máscara tenga valores 0 y 255
        if np.max(mascara) <= 1:  # Si los valores están en el rango [0, 1]
            print("Máscara en formato flotante [0, 1], convirtiendo a [0, 255]")
            mascara = (mascara * 255).astype(np.uint8)
        elif np.max(mascara) > 255:  # Si la máscara tiene valores fuera del rango esperado
            print("Máscara con valores fuera de rango [0, 255], normalizando")
            mascara = np.clip(mascara, 0, 255).astype(np.uint8)

        # Convertir la imagen a escala de grises
        imagen_gris = rgb2gray(imagen)
        
        # Extraer características Sobel para toda la imagen
        sobel_features = sobel(imagen_gris)
        
        # Extraer características HOG para toda la imagen
        hog_features, hog_image = hog(imagen_gris, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)

        # Calcular LBP para la imagen en escala de grises
        lbp = local_binary_pattern(imagen_gris, P=8, R=1, method='uniform')
        
        # Calcular los momentos de Hu
        imagen_byte = img_as_ubyte(imagen_gris)
        hu_moments = moments_hu(imagen_byte)
        
        # Identificar los píxeles de carretera y fondo
        indices_carretera = np.where(mascara == 255)  # Carretera
        indices_fondo = np.where(mascara == 0)      # Fondo

        # Número de píxeles de fondo que queremos seleccionar
        num_carretera = len(indices_carretera[0])
        num_fondo = len(indices_fondo[0])

        # Si los píxeles de fondo son más que los de carretera, hacer downsampling
        if num_fondo > num_carretera:
            downsampling_factor = num_fondo // num_carretera  # Ajustar el factor de downsampling
            indices_fondo_downsampled = indices_fondo[0][::downsampling_factor]  # Reducir los píxeles de fondo
        else:
            indices_fondo_downsampled = indices_fondo[0]  # Mantener todos los píxeles de fondo si son menores o iguales a los de carretera

        # Ahora aseguramos que los índices balanceados respeten las clases
        indices_balanceados = np.concatenate(( 
            # Índices de carretera con etiqueta 255
            np.array([255] * len(indices_carretera[0])), 
            # Índices de fondo balanceados con etiqueta 0
            np.array([0] * len(indices_fondo_downsampled))
        ))

        # Combinar los índices y las etiquetas para las características
        # Iterar sobre los índices balanceados
        for idx_pixel, etiqueta in zip(np.concatenate((indices_carretera[0], indices_fondo_downsampled)), indices_balanceados):
            i = idx_pixel // mascara.shape[1]  # Obtener la coordenada de la fila
            j = idx_pixel % mascara.shape[1]   # Obtener la coordenada de la columna

            # Extraer las características de Sobel para el píxel (i, j)
            sobel_value = sobel_features[i, j]
            
            # Calcular el índice del vector de características HOG correspondiente al píxel (i, j)
            hog_idx = (i // 16) * (imagen_gris.shape[1] // 16) + (j // 16)
            hog_value = hog_features[hog_idx]
            
            # Obtener el valor de LBP para el píxel (i, j)
            lbp_value = lbp[i, j]
            
            # Obtener los momentos de Hu (cada valor es un momento de Hu individual)
            hu_value = hu_moments.tolist()  # Momentos de Hu como lista
            
            # Concatenar las características de Sobel, HOG, LBP y Hu Moments
            caracteristicas_pixel = [sobel_value] + [hog_value] + [lbp_value] + hu_value
            
            # Escribir las características y la etiqueta en el CSV
            writer.writerow(caracteristicas_pixel + [etiqueta, imagen_idx])  # Aquí se usa imagen_idx para la imagen actual

# Verificar el contenido del CSV
with open(csv_file, mode='r') as file:
    reader = csv.reader(file)
    for i, row in enumerate(reader):
        if i < 5:  # Imprimir las primeras 5 filas
            print(row)

Se encontraron 20 pares de imágenes y máscaras.


Cargando imágenes y máscaras: 0it [00:00, ?it/s]c:\Users\Usuario\Desktop\proyecto_investigacion\env_pvc\lib\site-packages\skimage\feature\texture.py:360: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
Cargando imágenes y máscaras: 7it [00:48,  7.97s/it]

In [ ]:
import os
import numpy as np
import pandas as pd
import tifffile as tiff
from PIL import Image
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.filters import sobel
from skimage.feature import hog, local_binary_pattern
from skimage.measure import moments_hu
from skimage import img_as_ubyte
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Cargar el archivo CSV con las características y etiquetas
data = pd.read_csv("caracteristicas_imagenes.csv")

# Separar características y etiquetas
X = data[['sobel', 'hog', 'lbp', 'hu_moments_1', 'hu_moments_2', 'hu_moments_3', 'hu_moments_4', 'hu_moments_5', 'hu_moments_6', 'hu_moments_7']]
y = data['etiqueta']

# Dividir los datos en entrenamiento (80%), validación (10%) y prueba (10%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Entrenar el modelo Random Forest
clf = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=42, verbose=2, n_jobs=-1)
clf.fit(X_train, y_train)



In [ ]:
for imagen_idx, (imagen_nombre, mascara_nombre) in enumerate(pares_imagenes_mascaras):
    print("Ruta de la imagen:", os.path.join(directorio_satelital, imagen_nombre))
    print("Ruta de la máscara:", os.path.join(directorio_gt, mascara_nombre))


Ruta de la imagen: roads/sat_test\10078675_15.tiff
Ruta de la máscara: roads/gt\10078675_15.tif
Ruta de la imagen: roads/sat_test\10228675_15.tiff
Ruta de la máscara: roads/gt\10228675_15.tif
Ruta de la imagen: roads/sat_test\10228705_15.tiff
Ruta de la máscara: roads/gt\10228705_15.tif
Ruta de la imagen: roads/sat_test\10228720_15.tiff
Ruta de la máscara: roads/gt\10228720_15.tif
Ruta de la imagen: roads/sat_test\10228735_15.tiff
Ruta de la máscara: roads/gt\10228735_15.tif
Ruta de la imagen: roads/sat_test\10228750_15.tiff
Ruta de la máscara: roads/gt\10228750_15.tif
Ruta de la imagen: roads/sat_test\10378675_15.tiff
Ruta de la máscara: roads/gt\10378675_15.tif
Ruta de la imagen: roads/sat_test\10378690_15.tiff
Ruta de la máscara: roads/gt\10378690_15.tif
Ruta de la imagen: roads/sat_test\10378705_15.tiff
Ruta de la máscara: roads/gt\10378705_15.tif
Ruta de la imagen: roads/sat_test\10378720_15.tiff
Ruta de la máscara: roads/gt\10378720_15.tif
Ruta de la imagen: roads/sat_test\103787

In [ ]:
archivo_imagen = os.path.join(directorio_satelital, pares_imagenes_mascaras[imagen_idx][0])
if not os.path.exists(archivo_imagen):
    print(f"El archivo {archivo_imagen} no existe")
else:
    print(f"El archivo {archivo_imagen} está presente")


El archivo roads/sat_test\10528765_15.tiff no existe


In [245]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Cargar el CSV con las características
df = pd.read_csv("caracteristicas_imagenes_pixel_a_pixel.csv")

# Separar características y etiquetas
X = df[['R', 'G', 'B', 'sobel', 'hog']]  # Características (R, G, B, Sobel, HOG)
y = df['etiqueta']  # Etiquetas (carretera o fondo)

# Dividir los datos en entrenamiento (80%), validación (10%) y prueba (10%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Tamaño de entrenamiento: {X_train.shape[0]} muestras")
print(f"Tamaño de validación: {X_val.shape[0]} muestras")
print(f"Tamaño de prueba: {X_test.shape[0]} muestras")



# Entrenar el modelo Random Forest con los datos de entrenamiento
clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1,verbose=2)
clf.fit(X_train, y_train)

# Evaluar el rendimiento en el conjunto de validación
y_val_pred = clf.predict(X_val)
print("Informe de clasificación para el conjunto de validación:")
print(classification_report(y_val, y_val_pred))

# Evaluar el rendimiento en el conjunto de prueba
y_test_pred = clf.predict(X_test)
print("Informe de clasificación para el conjunto de prueba:")
print(classification_report(y_test, y_test_pred))



IndexError: index 255 is out of bounds for axis 0 with size 20

In [240]:
# Ejemplo con la última imagen de test:
imagen_idx = indices_test[0]  # Puedes elegir cualquier índice del conjunto de test
ruta_imagen = os.path.join(directorio_satelital, pares_imagenes_mascaras[imagen_idx][0])
imagen = imread(ruta_imagen)

# Generar la máscara predicha para la imagen
pred_mask = modelo.predict(X[indices_test == imagen_idx]).reshape(imagen.shape[0], imagen.shape[1])

# Mostrar la máscara predicha
plt.imshow(pred_mask, cmap='gray')
plt.title("Máscara Predicha")
plt.axis("off")
plt.show()

NameError: name 'indices_test' is not defined